### JSON Superset

CUE is a superset of JSON.
It adds the following conveniences:

- C-style comments,
- quotes may be omitted from field names without special characters,
- commas at the end of fields are optional,
- comma after last element in list is allowed,
- outer curly braces are optional.


In [3]:
cat 10_json.cue

one: 1
two: 2

// A field using quotes.
"two-and-a-half": 2.5

list: [
	1,
	2,
	3,
]


In [4]:
cue export 10_json.cue

{
    "list": [
        1,
        2,
        3
    ],
    "one": 1,
    "two": 2,
    "two-and-a-half": 2.5
}


## Types are values

CUE merges the concepts of values and types.
Below is a demonstration of this concept,
showing respectively
some data, a possible schema for this data,
and something in between: a typical CUE constraint.

### Data
```json
moscow: {
  name:    "Moscow"
  pop:     11.92M
  capital: true
}
```

### Schema
```json
municipality: {
  name:    string
  pop:     int
  capital: bool
}
```

### CUE
```json
largeCapital: {
  name:    string
  pop:     >5M
  capital: true
}
```

In general, in CUE one starts with a broad definition of a schema, describing all possible instances,
and then narrows down these definitions for particular use casesuntil a concrete data instance remains.

## Duplicate Fields

CUE allows duplicated field definitions as long as they don't conflict.

For values of basic types this means they must be equal.

For structs, fields are merged and duplicated fields are handled recursively.

For lists, all elements must match accordingly.

In [5]:
cat 30_duplicates.cue

a: 4
a: 4

s: { b: 2 }
s: { c: 2 }

l: [ 1, 2 ]
l: [ 1, 2 ]


In [6]:
cue export 30_duplicates.cue

{
    "a": 4,
    "s": {
        "b": 2,
        "c": 2
    },
    "l": [
        1,
        2
    ]
}


## Constraints

Constraints specify what values are allowed.
To CUE they are just values like anything else,
but conceptually they can be explained as something in between types and
concrete values.

But constraints can also reduce boilerplate.
If a constraint defines a concrete value, there is no need
to specify it in values to which this constraint applies.


In [7]:
cat 40_constraints_good.cue

schema: {
    name:  string
    age:   int
    human: true // always true
}

viola: schema
viola: {
    name: "Viola"
    age:  38
}


In [8]:
cue eval 40_constraints_good.cue

schema: {
    name:  string
    age:   int
    human: true
}
viola: {
    name:  "Viola"
    age:   38
    human: true
}


In [9]:
cat 40_constraints_bad.cue

schema: {
    name:  string
    age:   >12
}

viola: schema
viola: {
    name: "Viola"
    age:  11
}


In [11]:
cue eval 40_constraints_bad.cue || true

viola.age: invalid value 11 (out of bound >12):
    ./40_constraints_bad.cue:3:12
    ./40_constraints_bad.cue:9:11


### Definitions

In CUE, schemas are typically written as Definitions.
A definition is a field which identifier starts with
a `#` or `_#`.
This tells CUE that they are to be used for validation and should
not be output as data; it is okay for them to remain unspecified.

A definition also tells CUE the full set of allowed fields.
In other words, definitions define "closed" structs.
Including a `...` in struct keeps it open.


In [12]:
cat 43_definitions.cue

#Conn: {
    address:  string
    port:     int
    protocol: string
    // ...    // uncomment this to allow any field
}

lossy: #Conn & {
    address:  "1.2.3.4"
    port:     8888
    protocol: "udp"
    // foo: 2 // uncomment this to get an error
}


In [13]:
cue export 43_definitions.cue

{
    "lossy": {
        "address": "1.2.3.4",
        "port": 8888,
        "protocol": "udp"
    }
}


### Validation

Constraints can be used to validate values of concrete instances.
They can be applied to CUE data, or directly to YAML or JSON.

In [15]:
cat 47_validation.cue

#Language: {
	tag:  string
	name: =~"^\\p{Lu}" // Must start with an uppercase letter.
}
languages: [...#Language]


In [16]:
cat 47_validation_data.yaml

languages:
  - tag: en
    name: English
  - tag: nl
    name: dutch
  - tag: no
    name: Norwegian


In [17]:
cue vet 47_validation.cue 47_validation_data.yaml || true

languages.2.tag: conflicting values string and false (mismatched types string and bool):
    ./47_validation.cue:2:8
    ./47_validation_data.yaml:6:11
languages.1.name: invalid value "dutch" (does not match =~"^\\p{Lu}"):
    ./47_validation.cue:3:8
    ./47_validation_data.yaml:5:12


### Order is irrelevant

CUE's basic operations are defined in a way that the order in which
you combine two configurations is irrelevant to the outcome.

This is crucial property of CUE
that makes it easy for humans _and_ machines to reason over values and makes advanced tooling and automation possible.

In [18]:
cat 50_order.cue

a: {x: 1, y: int}
a: {x: int, y: 2}

b: {x: int, y: 2}
b: {x: 1, y: int}

In [19]:
cue eval 50_order.cue

a: {
    x: 1
    y: 2
}
b: {
    x: 1
    y: 2
}


### Folding of Single-Field Structs

In JSON, one defines nested values, one value at a time.
Another way to look at this is that a JSON configuration is a set of
path-value pairs.

In CUE one defines a set of paths to which to apply
a concrete value or constraint all at once.
Because of CUE's order independence, values get merged

This example shows some path-value pairs, as well as
a constraint that is applied to those to validate them.

In [20]:
cat 55_fold.cue

// path-value pairs
outer: middle1: inner: 3
outer: middle2: inner: 7

// collection-constraint pair
outer: [string]: inner: int


In [21]:
cue export 55_fold.cue

{
    "outer": {
        "middle1": {
            "inner": 3
        },
        "middle2": {
            "inner": 7
        }
    }
}


### Boilerplate

Constraints specify what values are allowed.
To CUE they are just values like anything else,
but conceptually they can be explained as something in between types and
concrete values.

Constraints can be used to validate values of concrete instances.
They can be applied to CUE data, or directly to YAML or JSON.

But constraints can also reduce boilerplate.
If a constraints defines a concrete value, there is no need
to specify it in values to which this constraint applies.

### More examples

Please see official introduction: https://cuelang.org/docs/tutorials/